![](https://cienciadosdados.com/images/2023/08/dsmm.png)

## Bibliotecas

In [2]:
import calendar
import locale
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import silhouette_score
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score

# Set the locale to use Monday as the first day of the week
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

# Set the first weekday to Monday (0 for Monday, 6 for Sunday)
calendar.setfirstweekday(calendar.MONDAY)

## Carregar Arquivos


In [3]:
#Itens vendidos 
df_item_sales = pd.read_csv("item_sales.csv")
#Preco dos itens 
df_item_prices = pd.read_csv("item_prices.csv")
#Calendario 
df_calendario = pd.read_csv("daily_calendar_with_events.csv")
#Base_mensal
df_base_mensal= pd.read_csv('base_mensal.csv')

## 1° Análise Exploratória

### DF Item Sales


In [4]:
df_item_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [5]:
df_item_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


In [6]:
# Realizando o "melt" para transformar o df_item_sales de formato largo para longo
df_item_sales_melted = df_item_sales.melt(id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'],
                      var_name='d',
                      value_name='sales')

In [7]:
df_item_sales_melted.head(10)

,id,item,category,department,store,store_code,region,d,sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
5,ACCESORIES_1_006_NYC_1,ACCESORIES_1_006,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
6,ACCESORIES_1_007_NYC_1,ACCESORIES_1_007,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
7,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12
8,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2
9,ACCESORIES_1_010_NYC_1,ACCESORIES_1_010,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0


In [8]:
# Primeiro, junte df_item_sales_melted com df_calendario para obter a data correspondente a cada 'd'
df_item_sales_with_dates = pd.merge(df_item_sales_melted, df_calendario[['d', 'date']], on='d', how='left')

# Converta a coluna 'date' para datetime
df_item_sales_with_dates['date'] = pd.to_datetime(df_item_sales_with_dates['date'])

# Crie uma coluna 'year_month' que representa o ano e o mês
df_item_sales_with_dates['year_month'] = df_item_sales_with_dates['date'].dt.to_period('M')

In [9]:

# Agora, agrupe por 'year_month' e some as vendas
monthly_sales = df_item_sales_with_dates.groupby(['year_month', 'id', 'item', 'category', 'department', 'store', 'store_code', 'region'])['sales'].sum().reset_index()

# Exibir as primeiras linhas do resultado
monthly_sales.head()


,year_month,id,item,category,department,store,store_code,region,sales
0,2011-01,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0
1,2011-01,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston,0
2,2011-01,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay,BOS_3,Boston,0
3,2011-01,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0
4,2011-01,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Harlem,NYC_2,New York,0


In [10]:
monthly_sales.year_month.max()

Period('2016-04', 'M')

In [11]:
# df_final = pd.melt(df_item_sales, id_vars=['id','item','category','department','store','store_code', 'region'],\
#                    var_name='d', value_name='sold_units')

### DF Calendario

In [12]:


# Convert 'date' to datetime
df_calendario['date'] = pd.to_datetime(df_calendario['date'])

# Convert 'date' to yearweek format
df_calendario['yearweek'] = df_calendario['date'].dt.strftime('%Y-W%U')

# Display the updated DataFrame
print(df_calendario)

           date    weekday  weekday_int       d event  yearweek
0    2011-01-29   Saturday            1     d_1   NaN  2011-W04
1    2011-01-30     Sunday            2     d_2   NaN  2011-W05
2    2011-01-31     Monday            3     d_3   NaN  2011-W05
3    2011-02-01    Tuesday            4     d_4   NaN  2011-W05
4    2011-02-02  Wednesday            5     d_5   NaN  2011-W05
...         ...        ...          ...     ...   ...       ...
1908 2016-04-20  Wednesday            5  d_1909   NaN  2016-W16
1909 2016-04-21   Thursday            6  d_1910   NaN  2016-W16
1910 2016-04-22     Friday            7  d_1911   NaN  2016-W16
1911 2016-04-23   Saturday            1  d_1912   NaN  2016-W16
1912 2016-04-24     Sunday            2  d_1913   NaN  2016-W17

[1913 rows x 6 columns]


In [13]:
df_calendario['year_str'] = df_calendario['yearweek'].dropna().astype(str).str.slice(0, 4)
df_calendario['week_str'] = df_calendario['yearweek'].dropna().astype(str).str.slice(4, 6)

### DF Item Prices

In [14]:
df_item_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [15]:
df_item_prices.loc[149]

item          ACCESORIES_1_001
category            ACCESORIES
store_code               NYC_1
yearweek                   NaN
sell_price             11.1454
Name: 149, dtype: object

In [16]:
#Procurando os valores faltantes 
df_item_prices[df_item_prices.isna().any(axis=1)]

,item,category,store_code,yearweek,sell_price
149,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
150,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
151,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
152,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
153,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000


In [17]:
df_item_prices.query("(item == 'ACCESORIES_1_001') & (store_code == 'NYC_1')")

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858
...,...,...,...,...,...
152,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
153,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
154,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
155,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454


In [18]:
df_item_prices.query("(item == 'ACCESORIES_1_001') & (sell_price) & (store_code == 'NYC_1')").tail(53)

,item,category,store_code,yearweek,sell_price
104,ACCESORIES_1_001,ACCESORIES,NYC_1,201526.0,10.9858
105,ACCESORIES_1_001,ACCESORIES,NYC_1,201527.0,10.9858
106,ACCESORIES_1_001,ACCESORIES,NYC_1,201528.0,10.9858
107,ACCESORIES_1_001,ACCESORIES,NYC_1,201529.0,10.9858
108,ACCESORIES_1_001,ACCESORIES,NYC_1,201530.0,10.9858
109,ACCESORIES_1_001,ACCESORIES,NYC_1,201531.0,10.9858
110,ACCESORIES_1_001,ACCESORIES,NYC_1,201532.0,10.9858
111,ACCESORIES_1_001,ACCESORIES,NYC_1,201533.0,10.9858
112,ACCESORIES_1_001,ACCESORIES,NYC_1,201534.0,10.9858
113,ACCESORIES_1_001,ACCESORIES,NYC_1,201535.0,10.9858


In [19]:
df_item_prices.query("(item == 'ACCESORIES_1_001') & (sell_price == 11.15) & (store_code == 'NYC_1')").tail(20)

,item,category,store_code,yearweek,sell_price


In [20]:
#Filtrando os dados para saber quais datas estão faltando 
df_item_prices.query("item  == 'ACCESORIES_1_001' & sell_price  == 11.15 & store_code == 'NYC_1' ").tail(20)

,item,category,store_code,yearweek,sell_price


In [21]:
#Inputar dados faltante , como não tempos da semana 18 a 25 . criar lista com dados faltantes

datas_faltantes = [201618.0,201619.0,201620.0,201621.0,201622.0,201623.0,201624.0,201625.0]


In [22]:
#Descobrimos as datas faltantes , calculamos as datas usando o metodo fillna 
df_item_prices['yearweek'].fillna (pd.Series (datas_faltantes * (len(df_item_prices) // len(datas_faltantes) +1))[:len(df_item_prices)], inplace=True)

In [23]:
# Converta a coluna 'yearweek' para o tipo string
df_item_prices['yearweek'] = df_item_prices['yearweek'].astype(str)

# Extraia o ano e a semana em colunas separadas
df_item_prices['ano'] = df_item_prices['yearweek'].str[:4]
df_item_prices['semana'] = df_item_prices['yearweek'].str[4:]

# Converta as novas colunas para o tipo numérico, se necessário
df_item_prices['ano'] = pd.to_numeric(df_item_prices['ano'])

df_item_prices['semana'] = pd.to_numeric(df_item_prices['semana'])

In [24]:
df_item_prices['year_str'] = df_item_prices['yearweek'].dropna().astype(str).str.slice(0, 4)
df_item_prices['week_str'] = df_item_prices['yearweek'].dropna().astype(str).str.slice(4, 6)

In [25]:
def year_week_to_date(year, week):
    try:
        if pd.notna(year) and pd.notna(week):
            return pd.to_datetime(f"{year}W{week}1", format='%YW%U%w')
    except Exception as e:
        print(f"Error with year: {year}, week: {week}. Exception: {e}")
    return np.nan

# Apply the vectorized function to the DataFrame columns separately
df_item_prices['date'] = np.vectorize(year_week_to_date)(df_item_prices['year_str'], df_item_prices['week_str'])

# Create a column 'year_month' representing the year and month
df_item_prices['year_month'] = df_item_prices['date'].dt.to_period('M')

# Aggregate df_item_prices to get the mean price per 'year_month', 'item', 'category', and 'store_code'
df_item_prices_agg = df_item_prices.groupby(['year_month', 'item', 'category', 'store_code'])['sell_price'].mean().reset_index()
df_item_prices_agg.rename(columns={'sell_price': 'mean_price'}, inplace=True)

# Display the first few rows of the result
df_item_prices_agg.head()


,year_month,item,category,store_code,mean_price
0,2011-01,ACCESORIES_1_004,ACCESORIES,BOS_1,5.7722
1,2011-01,ACCESORIES_1_004,ACCESORIES,BOS_2,5.7722
2,2011-01,ACCESORIES_1_004,ACCESORIES,NYC_2,5.7722
3,2011-01,ACCESORIES_1_004,ACCESORIES,NYC_4,5.7722
4,2011-01,ACCESORIES_1_004,ACCESORIES,PHI_1,5.7722


In [26]:
# Junção dos dataframes

merged_df = pd.merge(monthly_sales, df_item_prices, on=['year_month', 'item', 'category', 'store_code'], how='left')

# Exibir as primeiras linhas do resultado
merged_df.head()

,year_month,id,item,category,department,store,store_code,region,sales,yearweek,sell_price,ano,semana,year_str,week_str,date
0,2011-01,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,2011-01,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,2011-01,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay,BOS_3,Boston,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,2011-01,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,2011-01,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Harlem,NYC_2,New York,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [35]:
# Making a Dataframe of the 10 most sold items
top10 = merged_df[['item', 'sales', 'store_code']].groupby('item').sum()
top10= top10.sort_values('sales', ascending=False).iloc[:10]
top10

,sales,store_code
item,,
SUPERMARKET_3_090,4434562,BOS_1BOS_2BOS_3NYC_1NYC_2NYC_3NYC_4PHI_1PHI_2P...
SUPERMARKET_3_586,4061282,BOS_1BOS_2BOS_3NYC_1NYC_2NYC_3NYC_4PHI_1PHI_2P...
SUPERMARKET_3_252,2490018,BOS_1BOS_2BOS_3NYC_1NYC_2NYC_3NYC_4PHI_1PHI_2P...
SUPERMARKET_3_555,2165971,BOS_1BOS_2BOS_3NYC_1NYC_2NYC_3NYC_4PHI_1PHI_2P...
SUPERMARKET_3_587,1746835,BOS_1BOS_2BOS_3NYC_1NYC_2NYC_3NYC_4PHI_1PHI_2P...
SUPERMARKET_3_714,1741222,BOS_1BOS_2BOS_3NYC_1NYC_2NYC_3NYC_4PHI_1PHI_2P...
SUPERMARKET_3_694,1718464,BOS_1BOS_2BOS_3NYC_1NYC_2NYC_3NYC_4PHI_1PHI_2P...
SUPERMARKET_3_226,1601641,BOS_1BOS_2BOS_3NYC_1NYC_2NYC_3NYC_4PHI_1PHI_2P...
SUPERMARKET_3_202,1299454,BOS_1BOS_2BOS_3NYC_1NYC_2NYC_3NYC_4PHI_1PHI_2P...


### DF Base Mensal

In [ ]:
df_base_mensal.category.value_counts()

In [ ]:
df_base_mensal.info()

In [ ]:
num_itens = df_base_mensal['item'].nunique()
print(f"Há {num_itens} produtos únicos no dataframe.")


In [ ]:


# 1. Preparação dos dados:

# Agregando dados por item
df_items = df_base_mensal.groupby('item').agg({
    'sales':'sum',
    'mean_price':'mean',
    'store':'nunique',
    'region':'nunique'
}).reset_index()

# Renomeando colunas para melhor clareza
df_items = df_items.rename(columns={
    'store': 'unique_stores',
    'region': 'unique_regions'
})

# Normalização dos dados
features_to_scale = ['sales', 'mean_price', 'unique_stores', 'unique_regions']
scaler = StandardScaler()
df_items_scaled = scaler.fit_transform(df_items[features_to_scale])

# 2. Determinar o número ideal de clusters:

inertia = []
for i in range(1, 11):  # Testando de 1 a 10 clusters
    kmeans = KMeans(n_clusters=i, random_state=0).fit(df_items_scaled)
    inertia.append(kmeans.inertia_)

# Plotando o Método do Cotovelo
plt.figure(figsize=(10,5))
plt.plot(range(1, 11), inertia, marker='o', linestyle='--')
plt.title('Método do Cotovelo para Itens')
plt.xlabel('Número de clusters')
plt.ylabel('Inertia')
plt.show()


In [ ]:
# Aplicar o K-means com 15 clusters:
kmeans_15 = KMeans(n_clusters=6, random_state=0).fit(df_items_scaled)
df_items['cluster'] = kmeans_15.labels_

# Agora, df_items tem uma nova coluna chamada 'cluster' que indica a qual dos 15 clusters cada item pertence.
df_items

In [ ]:


# Criando o scatter plot
fig = px.scatter(df_items,
                 x='sales',
                 y='mean_price',
                 color='cluster',
                 hover_data=['item'],
                 title="Distribuição dos Itens nos Clusters")

fig.show()


In [ ]:


# Calculando o coeficiente de silhueta
silhouette_avg = silhouette_score(df_items_scaled, df_items['cluster'])

print(f"O coeficiente de silhueta médio para os clusters é: {silhouette_avg:.2f}")


In [ ]:


# Criando o gráfico de dispersão 3D
fig = px.scatter_3d(df_items,
                    x='sales',
                    y='mean_price',
                    z='unique_stores',
                    color='cluster',
                    labels={'sales': 'Sales', 'mean_price': 'Mean Price', 'unique_stores': 'Unique Stores Sold In'},
                    title="Clusters de Itens em 3D",
                    color_continuous_scale=px.colors.qualitative.Set1,
                    hover_data=['item'])

fig.show()


### Lojas

In [ ]:


# 1. Preparação dos dados:
# Selecione as características desejadas: sales, mean_price, região e diversidade de produtos por loja.
df_stores = df_base_mensal.groupby(['store', 'region']).agg({'sales':'sum', 'mean_price':'mean', 'item':'nunique'}).reset_index()
df_stores = df_stores.rename(columns={'item': 'unique_products'})

# Codificação One-Hot para a coluna 'region'
df_region_encoded = pd.get_dummies(df_stores['region'], prefix='region')
df_stores = pd.concat([df_stores, df_region_encoded], axis=1)

# Normalização dos dados
features_to_scale = ['sales', 'mean_price', 'unique_products'] + df_region_encoded.columns.tolist()
scaler = StandardScaler()
df_stores_scaled = scaler.fit_transform(df_stores[features_to_scale])

# 2. Determinar o número ideal de clusters:
inertia = []
for i in range(1, 11):  # Testando de 1 a 10 clusters
    kmeans = KMeans(n_clusters=i, random_state=0).fit(df_stores_scaled)
    inertia.append(kmeans.inertia_)

# Plotando o Método do Cotovelo
plt.figure(figsize=(10,5))
plt.plot(range(1, 11), inertia, marker='o', linestyle='--')
plt.title('Método do Cotovelo')
plt.xlabel('Número de clusters')
plt.ylabel('Inertia')
plt.show()


In [ ]:
# 3. Aplicar o K-means com o número ideal de clusters:
kmeans = KMeans(n_clusters=4, random_state=0).fit(df_stores_scaled)
df_stores['cluster'] = kmeans.labels_

# Agora, df_stores tem uma nova coluna chamada 'cluster' que indica a qual cluster cada loja pertence.


In [ ]:
df_stores

In [ ]:


# Criando uma lista para armazenar os dados de cada cluster
data = []

# Iterar através de cada cluster e coletar os nomes das lojas
for cluster in df_stores['cluster'].unique():
    cluster_data = df_stores[df_stores['cluster'] == cluster]
    store_names = cluster_data['store'].tolist()

    # Adicionar os nomes das lojas ao gráfico de barras para esse cluster
    data.append(go.Bar(name=f'Cluster {cluster}', x=[f'Cluster {cluster}'] * len(store_names), y=store_names, orientation='h'))

# Criando o gráfico
fig = go.Figure(data=data)

# Configurações adicionais
fig.update_layout(title_text='Lojas em cada Cluster', barmode='stack', yaxis_categoryorder='total ascending')

fig.show()


In [ ]:
num_stores = df_base_mensal['store'].nunique()
print(f"Há {num_stores} lojas únicas no dataframe.")


In [ ]:


# Criando o gráfico de violino
fig = px.violin(df_stores, y="store", x="cluster", box=True, points="all", title="Distribuição de Lojas por Cluster")

# Mostrando o gráfico
fig.show()


In [ ]:


# ...

# Após aplicar o K-means:
silhouette_avg = silhouette_score(df_stores_scaled, df_stores['cluster'])

print(f"O coeficiente de silhueta médio é: {silhouette_avg:.2f}")


## <font color = blue >2. Previsão de Vendas

### Baseline: Previsão por Loja

In [ ]:


# Preparando os dados
df_base_mensal['year_month'] = pd.to_datetime(df_base_mensal['year_month'])
df_base_mensal = pd.get_dummies(df_base_mensal, columns=['category', 'store'])  # Apenas 'category' e 'store' para One-Hot

X = df_base_mensal.drop(columns=['sales', 'id', 'year_month', 'item', 'store_code'])

# Definindo o modelo
model = lgb.LGBMRegressor()

# Validação cruzada usando TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score)
mape_scorer = make_scorer(MAPE, greater_is_better=False)

mse_scores = cross_val_score(model, X, y, cv=tscv, scoring=mse_scorer)
rmse_scores = cross_val_score(model, X, y, cv=tscv, scoring=rmse_scorer)
mae_scores = cross_val_score(model, X, y, cv=tscv, scoring=mae_scorer)
r2_scores = cross_val_score(model, X, y, cv=tscv, scoring=r2_scorer)
mape_scores = cross_val_score(model, X, y, cv=tscv, scoring=mape_scorer)

print(f"MSE: {-np.mean(mse_scores)}")
print(f"RMSE: {-np.mean(rmse_scores)}")
print(f"MAE: {-np.mean(mae_scores)}")
print(f"R2: {np.mean(r2_scores)}")
print(f"MAPE: {-np.mean(mape_scores)}")


### Previsão por Produto e Loja - Categoria Supermarket

In [ ]:

df_base_mensal.info()

In [ ]:
# Agrupando o dataframe por produto e somando as vendas
product_sales = df_base_mensal.groupby('item').sum()['sales']

# Ordenando os produtos pelo total de vendas, em ordem decrescente
sorted_product_sales = product_sales.sort_values(ascending=False)

# Calculando o total cumulativo das vendas
cumulative_sales = sorted_product_sales.cumsum()

# Calculando 80% das vendas totais
sales_80_percent = df_base_mensal['sales'].sum() * 0.8

# Identificando os produtos que compõem os 80% das vendas
top_20_percent_products = cumulative_sales[cumulative_sales <= sales_80_percent].index.tolist()

print(f"Os 20% dos produtos que geram 80% das vendas são: {top_20_percent_products}")


In [ ]:
top_20_percent_products

In [ ]:


# Função para calcular o MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Agrupando e filtrando os dados
product_sales = df_base_mensal.groupby('item').sum()['sales']
sorted_product_sales = product_sales.sort_values(ascending=False)
cumulative_sales = sorted_product_sales.cumsum()
sales_80_percent = df_base_mensal['sales'].sum() * 0.8
top_20_percent_products = cumulative_sales[cumulative_sales <= sales_80_percent].index.tolist()
df_supermarket = df_base_mensal[(df_base_mensal['item'].isin(top_20_percent_products)) & (df_base_mensal['category'] == 'SUPERMARKET')]

# Parâmetros para o RandomizedSearchCV
param_dist = {
    'objective': ['regression'],
    'boosting_type': ['gbdt', 'dart'],
    'learning_rate': [0.01, 0.02],
    'n_estimators': [40, 60],
    'seed': [0],
}

model = lgb.LGBMRegressor(verbose=-1)
tscv = TimeSeriesSplit(n_splits=4)

unique_items = df_supermarket['item'].unique()
unique_stores = df_supermarket['store'].unique()

results = []
r2_results = []
all_y = []
all_y_pred = []

for item in unique_items:
    for store in unique_stores:
        subset = df_supermarket[(df_supermarket['item'] == item) & (df_supermarket['store'] == store)]
        X = subset.drop(columns=['sales', 'year_month', 'id', 'item', 'store', 'category'])
        y = subset['sales']

        X = pd.get_dummies(X)
        X = X.replace([np.inf, -np.inf], np.nan)
        X.dropna(axis=1, inplace=True)

        constant_columns = X.columns[X.nunique() == 1]
        X = X.drop(columns=constant_columns)

        if X.shape[1] == 0:
            print(f"No features left for Item {item} and Store {store} after preprocessing. Skipping...")
            continue

        random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=2, scoring='neg_mean_squared_error', cv=tscv, verbose=0)
        random_search.fit(X, y)
        y_pred = random_search.best_estimator_.predict(X)

        all_y.extend(y.tolist())
        all_y_pred.extend(y_pred.tolist())

        # Adicionando o resultado R2 à lista r2_results
        r2 = r2_score(y, y_pred)
        r2_results.append((f"Item {item}, Store {store}", r2))

        for real, predicted in zip(y, y_pred):
            results.append({
                'item': item,
                'store': store,
                'real_value': real,
                'predicted_value': predicted
            })

# Calculando métricas agregadas

mae_general = mean_absolute_error(all_y, all_y_pred)
r2_general = r2_score(all_y, all_y_pred)
rmse_general = np.sqrt(mean_squared_error(all_y, all_y_pred))
metrics_data = {

    'MAE': [mae_general],
    'R2': [r2_general],
    'RMSE': [rmse_general]
}

# Salvando os resultados no Excel
with pd.ExcelWriter('results.xlsx') as writer:
    # Adicionando colunas de erro absoluto e erro percentual ao DataFrame
    df_real_predicted = pd.DataFrame(results)
    df_real_predicted['Abs_Error'] = abs(df_real_predicted['real_value'] - df_real_predicted['predicted_value'])
    df_real_predicted['Percentage_Error'] = (df_real_predicted['Abs_Error'] / df_real_predicted['real_value']) * 100

    # Salvando valores reais, preditos e erros em uma folha
    df_real_predicted.to_excel(writer, sheet_name='Real_vs_Predicted', index=False)


    # Salvando métricas agregadas em outra folha
    df_metrics = pd.DataFrame(metrics_data)
    df_metrics.to_excel(writer, sheet_name='Aggregated_Metrics', index=False)

    # Salvando modelos e R2 em outra folha
    df_r2 = pd.DataFrame(sorted(r2_results, key=lambda x: x[1], reverse=True), columns=['Model', 'R2'])
    df_r2.to_excel(writer, sheet_name='Model_R2', index=False)
